<a href="https://colab.research.google.com/github/komo135/tradingrl/blob/master/neuro_evolution_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Google ドライブをマウントするには、このセルを実行してください。
from google.colab import drive
drive.mount('/content/drive')

%cd drive/My Drive

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/My Drive


In [2]:
!pip install ta

In [0]:
import numpy as np
from scipy.special import expit
import pandas as pd
import pickle
from numba import jit as njit
from functools import lru_cache
from multiprocessing import Pool
import os
import time
import random
import ta
from net import *
from memory import *
from reward import *

In [0]:
class neuralnetwork:
    def __init__(self, id_, hidden_size = 128*1, window_size = 30, restore=False):
        if restore:
          f = open("./neuro_w.txt","rb")
          weights = pickle.load(f)
          self.W1 = weights[0]
          self.W2 = weights[1]
          self.W3 = np.random.randn(hidden_size, 3)
          # self.W3 = weights[2]
        else:
          self.W1 = np.random.randn(window_size, hidden_size)# / np.sqrt(window_size)
          self.W2 = np.random.randn(hidden_size, hidden_size)# / np.sqrt(hidden_size)
          self.W3 = np.random.randn(hidden_size, 3)# / np.sqrt(hidden_size)
        self.fitness_list = []
        self.fitness = 0
        self.id = id_

@njit
def gaussiandropout(w,rate,train=True):
  if train:
    if 0. < rate < 1.:
      scale = np.sqrt(rate / (1. - rate))
      w *= np.random.normal(1.,scale,w.shape)
  return w

# @njit
def sigmoid(x):
  x = 1 / (1 + expit(-x))
  return x

# @njit
def swish(x):
  x *= sigmoid(x)
  return x
 
def relu(X):
    return np.maximum(X, 0)
  
# @njit
def softmax(X):
    e_x = np.exp(X - np.max(X, axis=-1, keepdims=True))
    e_x /= np.sum(e_x, axis=-1, keepdims=True)
    return e_x
  
@njit
def feed_forward(X, nets,rate=0.2,train=True):
    X = X.flatten().reshape(1,-1)
    # w1 = gaussiandropout(nets.W1,rate,train)
    a1 = np.dot(X, nets.W1)
    z1 = swish(a1)
    a2 = np.dot(z1,nets.W2)
    z2 = swish(a2)
    a3 = np.dot(z2, nets.W3)
    return np.tanh(a3)

In [0]:
class NeuroEvolution:
    def __init__(self, population_size, mutation_rate, model_generator, state_size, window_size, path, step_size, spread=10, pip_cost=1000, los_cut=20,restore=False):
        self.population_size = population_size
        self.mutation_rate = mutation_rate
        self.mutation_rate_origin = mutation_rate
        self.model_generator = model_generator
        self.state_size = state_size
        self.window_size = window_size
        self.path = path
        self.step_size = step_size
        self.spread = spread / pip_cost
        self.pip_cost = pip_cost
        self.los_cut = los_cut
        self.restore = restore
        self.preproc()
        self.rewards = reward3
        self.fittest_individual = None
        
    def preproc(self):
          self.dat = df = pd.read_csv(self.path)
          s = np.asanyarray(ta.stoch(df["High"],df["Low"],df["Close"],14)).reshape((-1, 1)) - np.asanyarray(ta.stoch_signal(df["High"],df["Low"],df["Close"],14)).reshape((-1, 1))
          x = np.asanyarray(ta.daily_log_return(df["Close"])).reshape((-1,1))
          m = np.asanyarray(ta.macd_diff(df["Close"])).reshape((-1,1))
          # x = m
          x = np.concatenate([m,s,x], 1)
          y = np.asanyarray(self.dat[["Open"]])

          gen = tf.keras.preprocessing.sequence.TimeseriesGenerator(x, y, self.window_size)
          self.x = []
          self.y = []
          for i in gen:
              self.x.extend(i[0].tolist())
              self.y.extend(i[1].tolist())
          self.x = np.asanyarray(self.x)
          self.y = np.asanyarray(self.y)

          self.df = self.x[-self.step_size::]
          self.trend = self.y[-self.step_size::]

    def _initialize_population(self):
        self.population = []
        for i in range(self.population_size):
            self.population.append(self.model_generator(i,window_size = self.window_size*self.df.shape[-1], restore=self.restore))
    
    def mutate(self, individual, scale=1.0):
      if np.random.random() < 0.01:
        mutation_mask = np.random.binomial(1, p=self.mutation_rate, size=individual.W1.shape)
        individual.W1 += np.random.normal(loc=0, scale=scale, size=individual.W1.shape) * mutation_mask
        mutation_mask = np.random.binomial(1, p=self.mutation_rate, size=individual.W2.shape)
        individual.W2 += np.random.normal(loc=0, scale=scale, size=individual.W2.shape) * mutation_mask
        mutation_mask = np.random.binomial(1, p=self.mutation_rate, size=individual.W3.shape)
        individual.W3 += np.random.normal(loc=0, scale=scale, size=individual.W3.shape) * mutation_mask
      return individual
    
    def inherit_weights(self, parent, child):
        child.W1 = parent.W1.copy()
        child.W2 = parent.W2.copy()
        child.W3 = parent.W3.copy()
        return child
    
    def crossover(self, parent1, parent2):
        child1 = self.model_generator((parent1.id+1)*10)
        child1 = self.inherit_weights(parent1, child1)
        child2 = self.model_generator((parent2.id+1)*10)
        child2 = self.inherit_weights(parent2, child2)
        # first W
        n_neurons = child1.W1.shape[1]
        cutoff = np.random.randint(0, n_neurons)
        child1.W1[:, cutoff:] = parent2.W1[:, cutoff:].copy()
        child2.W1[:, cutoff:] = parent1.W1[:, cutoff:].copy()
        # second W
        n_neurons = child1.W2.shape[1]
        cutoff = np.random.randint(0, n_neurons)
        child1.W2[:, cutoff:] = parent2.W2[:, cutoff:].copy()
        child2.W2[:, cutoff:] = parent1.W2[:, cutoff:].copy()
        # third W
        n_neurons = child1.W3.shape[1]
        cutoff = np.random.randint(0, n_neurons)
        child1.W3[:, cutoff:] = parent2.W3[:, cutoff:].copy()
        child2.W3[:, cutoff:] = parent1.W3[:, cutoff:].copy()
        return child1, child2
    
    def act(self, p, state, rate=0.2,train=True):
        logits = feed_forward(state, p, rate, train)
        return np.argmax(logits, 1)[0]
#         return np.argmax(logits[0])
    
    def test(self, individual, i):
        states = []
        pip = []
        history = []
        h_p = []
        provisional_pip = []
        total_pip = 0.0
        position = 3
        h = np.random.randint(100,self.x.shape[0]-(self.step_size+1))
        self.df = self.x[h:h+self.step_size]
        self.trend = self.y[h:h+self.step_size]
        for t in range(0, len(self.trend) - 1):
            action = self.act(individual, self.df[t], train=False)
            history.append(action)
            states,provisional_pip,position,total_pip = self.rewards(self.trend[t],pip,provisional_pip,action,
                                                                     position,states,self.pip_cost,self.spread,total_pip,lc=self.los_cut/1000)
#             print(len(provisional_pip))
            h_p.append(position)
        fitness = total_pip * self.pip_cost
        self.pip = np.asanyarray(provisional_pip) * self.pip_cost
        self.pip = [p if p >= -self.los_cut else -self.los_cut for p in self.pip]
        self.total_pip = np.sum(self.pip)
        mean_pip = self.total_pip / (t + 1)
        trade_accuracy = np.mean(np.asanyarray(self.pip) > 0)
        self.trade = trade_accuracy
        mean_pip *= 96
        prob = self.prob(history)
        position_prob = self.prob(h_p)
      
        print("")
        print('action probability = ', prob)
        print("buy = ", position_prob[1], " sell = ", position_prob[-1])
        print('trade accuracy = ', trade_accuracy)
        print('epoch: %d, total rewards: %f, mean rewards: %f' % (i+1, float(self.total_pip), float(mean_pip)))
    
    def run_fit(self,i):
      accuracy = 0
      for _ in range(20):
        h = np.random.randint(self.x.shape[0]-(24+1))
        df = self.x[h:h+24]
        trend = self.y[h:h+24]
        states = []
        pip = []
        provisional_pip = []
        total_pip = 0.0
        position = 3
        for t in range(0, len(trend) - 1):
            action = self.act(i, df[t])
            states,provisional_pip,position,total_pip = self.rewards(trend[t],pip,provisional_pip,
                                                                      action,position,states,self.pip_cost,self.spread,total_pip,lc=self.los_cut/1000)
        accuracy += np.mean(np.array(provisional_pip) > 0) * 100
        
      return accuracy
    
    def calculate_fitness(self):
        p = Pool(os.cpu_count())
        # args = range(self.population_size)
        fitness_list = p.map(self.run_fit, self.population)
        p.close()
        for i in range(self.population_size):
          self.population[i].fitness = fitness_list[i]

    def prob(self,history):
        prob = np.asanyarray(history)
        a = np.mean(prob == 0)
        b = np.mean(prob == 1)
        c = 1 - (a + b)
        prob = [a,b,c]
        return prob
    
    def sort(self):
        self.fitnesses = fitnesses = [i.fitness for i in self.population]
        fitnesses = np.array(neural_evolve.fitnesses).flatten()
        self.sort_fitness = sort_fitness = np.argsort(neural_evolve.fitnesses,None)[::-1]
        self.population = [self.population[int(i)] for i in sort_fitness]
        self.fittest_individual = fittest_individual = self.population[0]

    def evolve(self, generations=20, checkpoint= 5):
        self._initialize_population()
        n_winners = int(self.population_size * 0.1)
        n_parents = self.population_size - n_winners
        for epoch in range(generations):
            if epoch % 10 == 0:
              self.change = True
            self.calculate_fitness()
            self.sort()
            if (epoch+1) % checkpoint == 0:
                self.test(self.fittest_individual, epoch)
                save = [self.fittest_individual.W1, self.fittest_individual.W2, self.fittest_individual.W3]
                f = open('neuro_w.txt', 'wb')
                # f2 = open('neuro_train.txt', 'wb')
                pickle.dump(save, f)
            next_population = [self.population[i] for i in range(n_winners)]
            total_fitness = np.sum([np.abs(i.fitness) for i in self.population])
            parent_probabilities = [np.abs(i.fitness / total_fitness) for i in self.population]
            sort_probabilities = np.argsort(parent_probabilities,None).reshape(-1,1)
            parent_probabilities = [parent_probabilities[int(i)] for i in sort_probabilities]
            parent_probabilities = np.array(parent_probabilities).flatten()
            parents = np.random.choice(self.population, size=n_parents, p=parent_probabilities, replace=False)
            for i in np.arange(0, len(parents), 2):
                child1, child2 = self.crossover(parents[i], parents[i+1])
                next_population += [self.mutate(child1), self.mutate(child2)]
            self.population = next_population


In [0]:
population_size = 100
generations = 100
mutation_rate = 0.1
window_size = 30
step_size = 24 * 20
los_cut = 300
restore = False
path = "audpred60.csv"

neural_evolve = NeuroEvolution(population_size, mutation_rate, neuralnetwork,
                              window_size, window_size, path, step_size,restore=restore, los_cut=los_cut)

In [0]:
fittest_nets = neural_evolve.evolve(50000,1)


action probability =  [0.5949895615866388, 0.29436325678496866, 0.11064718162839249]
buy =  0.6764091858037579  sell =  0.32359081419624214
trade accuracy =  0.391304347826087
epoch: 1, total rewards: -5237.000000, mean rewards: -1049.586639

action probability =  [0.697286012526096, 0.22338204592901878, 0.07933194154488521]
buy =  0.7244258872651357  sell =  0.2755741127348643
trade accuracy =  0.444015444015444
epoch: 2, total rewards: -3847.000000, mean rewards: -771.006263

action probability =  [0.7766179540709812, 0.13778705636743216, 0.08559498956158662]
buy =  0.8288100208768268  sell =  0.17118997912317324
trade accuracy =  0.4703389830508475
epoch: 3, total rewards: 15585.000000, mean rewards: 3123.507307

action probability =  [0.6137787056367432, 0.2881002087682672, 0.09812108559498967]
buy =  0.6910229645093946  sell =  0.3089770354906054
trade accuracy =  0.4495114006514658
epoch: 4, total rewards: 12762.000000, mean rewards: 2557.728601

action probability =  [0.4801670